# Stock News Analysis with AI Scoring (Amazon Bedrock)

This notebook:
1. Fetches news for a stock ticker from Yahoo Finance (last 3 months) using defeatbeta-api
2. Uses Amazon Bedrock AI to select the most interesting news items
3. Fetches full content using defeatbeta-api
4. Scores each news item on multiple criteria using Amazon Bedrock

## 1. Install Required Libraries

In [51]:
!pip install defeatbeta-api boto3 matplotlib pandas -q

## 2. Import Libraries

In [52]:
import boto3
import json
import os
from datetime import datetime, timedelta
import pandas as pd
from typing import List, Dict, Any
import time
import numpy as np
import matplotlib.pyplot as plt

# Import defeatbeta-api
from defeatbeta_api.data.ticker import Ticker

## 3. Configuration

In [53]:
# Configuration
STOCK_TICKER = "AAPL"  # Change this to your desired stock ticker
TOP_N_NEWS = 10  # Number of most interesting news to analyze
MONTHS_BACK = 3  # How many months of news to fetch

# AWS Bedrock Configuration
AWS_REGION = os.environ.get("AWS_REGION", "us-east-1")  # Change to your preferred region
BEDROCK_MODEL_ID = "anthropic.claude-3-5-sonnet-20241022-v2:0"  # Claude 3.5 Sonnet

# Initialize Bedrock client
bedrock_runtime = boto3.client(
    service_name='bedrock-runtime',
    region_name=AWS_REGION
)

print(f"Using Amazon Bedrock in region: {AWS_REGION}")
print(f"Model: {BEDROCK_MODEL_ID}")
print(f"Analyzing stock: {STOCK_TICKER}")

Using Amazon Bedrock in region: us-west-2
Model: anthropic.claude-3-5-sonnet-20241022-v2:0
Analyzing stock: AAPL


## 4. Bedrock Helper Functions

In [54]:
def invoke_bedrock_model(prompt: str, max_tokens: int = 2000) -> str:
    """
    Invoke Amazon Bedrock model with a prompt.
    
    Args:
        prompt: The prompt to send to the model
        max_tokens: Maximum tokens in response
    
    Returns:
        Model response text
    """
    # Prepare the request body for Claude 3.5 Sonnet
    body = json.dumps({
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": max_tokens,
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ],
        "temperature": 0.7,
        "top_p": 0.9
    })
    
    try:
        # Invoke the model
        response = bedrock_runtime.invoke_model(
            modelId=BEDROCK_MODEL_ID,
            body=body
        )
        
        # Parse response
        response_body = json.loads(response['body'].read())
        return response_body['content'][0]['text']
    
    except Exception as e:
        print(f"Error invoking Bedrock: {e}")
        raise

## 5. Fetch News from DefeatBeta API

In [55]:
def get_stock_news(ticker_symbol: str, months_back: int = 3) -> List[Dict[str, Any]]:
    """
    Fetch news for a given ticker using defeatbeta-api.
    
    Args:
        ticker_symbol: Stock ticker symbol
        months_back: Number of months to look back
    
    Returns:
        List of news items with title, content, publisher, and timestamp
    """
    print(f"Fetching news for {ticker_symbol}...")
    
    # Initialize ticker
    ticker = Ticker(ticker_symbol)
    
    # Get news data
    news_data = ticker.news()
    
    # Get the news list as DataFrame
    news_df = news_data.get_news_list()
    
    if news_df is None or len(news_df) == 0:
        print("No news found for this ticker")
        return []
    
    # Filter news from last N months
    cutoff_date = datetime.now() - timedelta(days=months_back * 30)
    
    filtered_news = []
    for _, row in news_df.iterrows():
        try:
            # Parse the report date
            report_date = pd.to_datetime(row['report_date'])

            if report_date >= cutoff_date:
                news_item = {
                    'uuid': row['uuid'],
                    'title': row['title'],
                    'publisher': row['publisher'],
                    'report_date': report_date.strftime('%Y-%m-%d'),
                    'link': row['link'],
                    'content': ''  # Will be populated later
                }
                filtered_news.append(news_item)
        except Exception as e:
            print(f"Error processing news item: {e}")
            continue
    
    print(f"Found {len(filtered_news)} news items from the last {months_back} months")
    return filtered_news

In [ ]:
# Fetch news
# news_items = get_stock_news(STOCK_TICKER, months_back=MONTHS_BACK)

# # Display sample
# print("\nSample news items:")
# for i, item in enumerate(news_items[:3]):
#     print(f"\n{i+1}. {item['title']}")
#     print(f"   Publisher: {item['publisher']}")
#     print(f"   Date: {item['report_date']}")
#     print(f"   Link: {item['link'][:80]}..." if len(item['link']) > 80 else f"   Link: {item['link']}")

Fetching news for AAPL...


Found 894 news items from the last 3 months

Sample news items:

1. IBN Initiates Coverage of Wearable Devices Ltd. (NASDAQ: WLDS)
   Publisher: GlobeNewswire
   Date: 2025-08-05
   Link: https://finance.yahoo.com/news/ibn-initiates-coverage-wearable-devices-123000448...

2. Palantir’s Ultra-Expensive Valuation Sparks Worry Into Results
   Publisher: Bloomberg
   Date: 2025-08-05
   Link: https://finance.yahoo.com/news/palantir-ultra-expensive-valuation-sparks-1030007...

3. 1 S&P 500 Stock to Target This Week and 2 We Find Risky
   Publisher: StockStory
   Date: 2025-08-05
   Link: https://finance.yahoo.com/news/1-p-500-stock-target-043406147.html


## 6. AI Selection of Most Interesting News

In [57]:
def select_interesting_news(news_items: List[Dict], ticker: str, top_n: int = 10) -> List[Dict]:
    """
    Use Amazon Bedrock AI to select the most interesting news items.
    
    Args:
        news_items: List of news items
        ticker: Stock ticker symbol
        top_n: Number of top news to select
    
    Returns:
        List of selected news items with rankings
    """
    if len(news_items) == 0:
        return []
    
    print(f"\nUsing Amazon Bedrock AI to select top {top_n} most interesting news items...")
    
    # Prepare news list for AI
    news_list_text = "\n".join([
        f"{i+1}. [{item['report_date']}] {item['title']} (Publisher: {item['publisher']})"
        for i, item in enumerate(news_items)
    ])
    
    prompt = f"""You are a financial analyst tasked with selecting the most interesting and impactful news about {ticker}.

Here are the news items:

{news_list_text}

Please analyze these news items and select the top {min(top_n, len(news_items))} most interesting ones based on:
- Potential impact on stock price
- Relevance to company fundamentals
- Newsworthiness and significance
- Market-moving potential

Return ONLY a JSON array with the indices (1-based) of the selected news items in order of importance, like this:
[1, 5, 3, 8, 2, 12, 7, 9, 4, 11]

No explanations, just the JSON array."""
    
    try:
        response_text = invoke_bedrock_model(prompt, max_tokens=500)
        
        # Parse response
        content = response_text.strip()
        # Extract JSON array from response
        if '[' in content and ']' in content:
            start = content.index('[')
            end = content.rindex(']') + 1
            content = content[start:end]
        
        selected_indices = json.loads(content)
        
        # Get selected news items
        selected_news = [news_items[idx - 1] for idx in selected_indices if 0 < idx <= len(news_items)]
        
        print(f"Selected {len(selected_news)} news items")
        return selected_news[:top_n]
    
    except Exception as e:
        print(f"Error selecting news: {e}")
        print("Returning first {top_n} items as fallback")
        return news_items[:top_n]

In [ ]:
# Select interesting news
# interesting_news = select_interesting_news(news_items, STOCK_TICKER, top_n=TOP_N_NEWS)

# print("\nSelected interesting news:")
# for i, item in enumerate(interesting_news):
#     print(f"\n{i+1}. {item['title']}")
#     print(f"   Date: {item['report_date']}")


Using Amazon Bedrock AI to select top 10 most interesting news items...
Selected 10 news items

Selected interesting news:

1. Apple Jumps 5.1% After $100 Billion U.S. Manufacturing Pledge
   Date: 2025-08-07

2. Apple announces additional $100 billion in US investment, following Trump iPhone tariff threat
   Date: 2025-08-07

3. Apple Raises US Investment to $600 Billion as Trump Eyes 100% Chip Tariffs
   Date: 2025-08-07

4. Apple Announces $100 Billion US Investment Ahead of Trump Event
   Date: 2025-08-07

5. Apple Stock Rallies on Trump’s Plan for ‘Make in America’ Tariff Exemptions
   Date: 2025-08-08

6. Apple's $600 Billion U.S. Investment Could Reshape Its Future
   Date: 2025-08-10

7. Apple announces five-year partnership as U.S. broadcast partner for Formula 1
   Date: 2025-10-18

8. Apple (AAPL) Q4 Earnings and Revenues Beat Estimates
   Date: 2025-10-31

9. Market Chatter: Globalstar Explores Sale, Holds Early Talks With SpaceX
   Date: 2025-10-31

10. Heard on the Stree

## 7. Fetch Full Content from DefeatBeta API

In [59]:
def fetch_news_content(ticker_symbol: str, news_items: List[Dict]) -> List[Dict]:
    """
    Fetch full content for all news items using defeatbeta-api.
    
    Args:
        ticker_symbol: Stock ticker symbol
        news_items: List of news items with UUIDs
    
    Returns:
        List of news items with full content added
    """
    print("\nFetching full content for selected news...")
    
    # Initialize ticker
    ticker = Ticker(ticker_symbol)
    news_api = ticker.news()
    
    for i, item in enumerate(news_items):
        print(f"Fetching content {i+1}/{len(news_items)}...")
        try:
            # Get full news content using UUID
            news_detail = news_api.get_news(item['uuid'])
            
            if news_detail is not None and not news_detail.empty:
                # Extract content from the DataFrame
                content = news_detail.iloc[0]['content'] if 'content' in news_detail.columns else ''
                item['content'] = content if content else item['title']
            else:
                item['content'] = item['title']  # Fallback to title if no content
                
        except Exception as e:
            print(f"Error fetching content for item {i+1}: {e}")
            item['content'] = item['title']  # Fallback to title
        
        time.sleep(0.5)  # Rate limiting
    
    print("Content fetching complete!")
    return news_items

In [ ]:
# Fetch full content
# news_with_content = fetch_news_content(STOCK_TICKER, interesting_news)

# # Display sample content
# print("\nSample content preview:")
# if len(news_with_content) > 0 and news_with_content[0]['content']:
#     print(f"Title: {news_with_content[0]['title']}")
#     print(f"Content preview: {news_with_content[0]['content'][:300]}...")
# else:
#     print("No content available for first item")


Fetching full content for selected news...
Fetching content 1/10...
Fetching content 2/10...
Fetching content 3/10...
Fetching content 4/10...
Fetching content 5/10...
Fetching content 6/10...
Fetching content 7/10...
Fetching content 8/10...
Fetching content 9/10...
Fetching content 10/10...
Content fetching complete!

Sample content preview:
Title: Apple Jumps 5.1% After $100 Billion U.S. Manufacturing Pledge
Content preview: Apple Jumps 5.1% After $100 Billion U.S. Manufacturing Pledge...


## 8. AI Scoring of News Items

In [61]:
def score_news_item(news_item: Dict, ticker: str) -> Dict:
    """
    Score a news item on multiple criteria using Amazon Bedrock AI.
    
    Args:
        news_item: News item with title, content, etc.
        ticker: Stock ticker symbol
    
    Returns:
        Dictionary with scores and explanation
    """
    content_text = news_item['content'] if news_item['content'] else news_item['title']
    
    prompt = f"""You are a financial analyst scoring news about {ticker}.

News Title: {news_item['title']}
Published: {news_item['report_date']}
Publisher: {news_item['publisher']}

Content:
{content_text[:3000]}

Please analyze this news and score it on the following criteria (0-10 scale):

1. SENTIMENT SCORE (0-10):
   - 0 = Very negative for stock
   - 5 = Neutral
   - 10 = Very positive for stock

2. LIKELIHOOD SCORE (0-10):
   - 0 = Pure speculation/rumor
   - 5 = Possible/probable
   - 10 = Already happened/100% certain

3. IMPACT SCORE (0-10):
   - 0 = No impact on stock price
   - 5 = Moderate impact
   - 10 = Major impact on stock price

Return ONLY a JSON object with this exact structure:
{{
  "sentiment_score": <number>,
  "likelihood_score": <number>,
  "impact_score": <number>,
  "explanation": "<brief 2-3 sentence explanation of the scores>"
}}

No other text, just the JSON object."""
    
    try:
        response_text = invoke_bedrock_model(prompt, max_tokens=1000)
        content = response_text.strip()
        
        # Extract JSON from response
        if '{' in content and '}' in content:
            start = content.index('{')
            end = content.rindex('}') + 1
            content = content[start:end]
        
        scores = json.loads(content)
        return scores
    except Exception as e:
        print(f"Error scoring news: {e}")
        return {
            "sentiment_score": 5,
            "likelihood_score": 5,
            "impact_score": 5,
            "explanation": "Error occurred during scoring"
        }


def score_all_news(news_items: List[Dict], ticker: str) -> List[Dict]:
    """
    Score all news items using Amazon Bedrock.
    
    Args:
        news_items: List of news items
        ticker: Stock ticker symbol
    
    Returns:
        List of news items with scores added
    """
    print("\nScoring news items with Amazon Bedrock AI...")
    
    for i, item in enumerate(news_items):
        print(f"Scoring news {i+1}/{len(news_items)}...")
        scores = score_news_item(item, ticker)
        item['scores'] = scores
        time.sleep(1)  # Rate limiting
    
    print("Scoring complete!")
    return news_items

In [ ]:
# Score all news
# scored_news = score_all_news(news_with_content, STOCK_TICKER)


Scoring news items with Amazon Bedrock AI...
Scoring news 1/10...
Scoring news 2/10...
Scoring news 3/10...
Scoring news 4/10...
Scoring news 5/10...
Scoring news 6/10...
Scoring news 7/10...
Scoring news 8/10...
Scoring news 9/10...
Scoring news 10/10...
Scoring complete!


## 9. Display Results

In [ ]:
def display_scored_news(news_items: List[Dict]):
    """
    Display scored news in a formatted way.
    """
    print("\n" + "="*100)
    print(f"STOCK NEWS ANALYSIS FOR {STOCK_TICKER}")
    print("="*100 + "\n")
    
    for i, item in enumerate(news_items):
        scores = item.get('scores', {})
        
        print(f"\n{'='*100}")
        print(f"NEWS #{i+1}")
        print(f"{'='*100}")
        print(f"\nTitle: {item['title']}")
        print(f"Publisher: {item['publisher']}")
        print(f"Date: {item['report_date']}")
        print(f"Link: {item['link']}")
        print(f"\n{'─'*100}")
        print("SCORES:")
        print(f"{'─'*100}")
        
        sentiment = scores.get('sentiment_score', 5)
        likelihood = scores.get('likelihood_score', 5)
        impact = scores.get('impact_score', 5)
        
        sentiment_label = 'Negative' if sentiment < 4 else 'Neutral' if sentiment <= 6 else 'Positive'
        likelihood_label = 'Speculation' if likelihood < 4 else 'Probable' if likelihood <= 7 else 'Confirmed'
        impact_label = 'Low Impact' if impact < 4 else 'Moderate' if impact <= 6 else 'High Impact'
        
        print(f"  📊 Sentiment Score:   {sentiment}/10  ({sentiment_label})")
        print(f"  🎯 Likelihood Score:  {likelihood}/10  ({likelihood_label})")
        print(f"  💥 Impact Score:      {impact}/10  ({impact_label})")
        print(f"\n  Explanation: {scores.get('explanation', 'N/A')}")
        print()

# display_scored_news(scored_news)


STOCK NEWS ANALYSIS FOR AAPL


NEWS #1

Title: Apple Jumps 5.1% After $100 Billion U.S. Manufacturing Pledge
Publisher: Zacks
Date: 2025-08-07
Link: https://finance.yahoo.com/news/apple-jumps-5-1-100-121100958.html

────────────────────────────────────────────────────────────────────────────────────────────────────
SCORES:
────────────────────────────────────────────────────────────────────────────────────────────────────
  📊 Sentiment Score:   8/10  (Positive)
  🎯 Likelihood Score:  10/10  (Confirmed)
  💥 Impact Score:      7/10  (High Impact)

  Explanation: The $100B manufacturing commitment demonstrates strong confidence in US operations and likely appeals to both investors and policymakers. The 5.1% stock jump shows immediate positive market reaction. The large investment signals long-term growth plans while potentially reducing geopolitical risks associated with overseas manufacturing.


NEWS #2

Title: Apple announces additional $100 billion in US investment, following Trump iP

## 10. Create Summary DataFrame

In [ ]:
# Create a DataFrame for easy analysis
# summary_data = []
# for item in scored_news:
#     scores = item.get('scores', {})
#     summary_data.append({
#         'Title': item['title'][:80] + '...' if len(item['title']) > 80 else item['title'],
#         'Date': item['report_date'],
#         'Publisher': item['publisher'],
#         'Sentiment': scores.get('sentiment_score', 'N/A'),
#         'Likelihood': scores.get('likelihood_score', 'N/A'),
#         'Impact': scores.get('impact_score', 'N/A'),
#         'Link': item['link']
#     })

# df = pd.DataFrame(summary_data)
# print("\nSummary DataFrame:")
# df


Summary DataFrame:


,Title,Date,Publisher,Sentiment,Likelihood,Impact,Link
0,Apple Jumps 5.1% After $100 Billion U.S. Manuf...,2025-08-07,Zacks,8,10,7,https://finance.yahoo.com/news/apple-jumps-5-1...
1,Apple announces additional $100 billion in US ...,2025-08-07,Yahoo Finance,7,8,8,https://finance.yahoo.com/news/apple-announces...
2,Apple Raises US Investment to $600 Billion as ...,2025-08-07,MT Newswires,6,8,7,https://finance.yahoo.com/news/apple-raises-us...
3,Apple Announces $100 Billion US Investment Ahe...,2025-08-07,Bloomberg,8,7,7,https://finance.yahoo.com/news/trump-apple-ann...
4,Apple Stock Rallies on Trump’s Plan for ‘Make ...,2025-08-08,The Wall Street Journal,8,5,7,https://finance.yahoo.com/m/1c152b80-e9f6-3122...
5,Apple's $600 Billion U.S. Investment Could Res...,2025-08-10,Motley Fool,8,5,7,https://finance.yahoo.com/m/a4fd4c66-a9bf-369a...
6,Apple announces five-year partnership as U.S. ...,2025-10-18,TipRanks,7,10,4,https://finance.yahoo.com/m/aa6d4ee1-1bb9-3678...
7,Apple (AAPL) Q4 Earnings and Revenues Beat Est...,2025-10-31,Zacks,8,10,7,https://finance.yahoo.com/news/apple-aapl-q4-e...
8,"Market Chatter: Globalstar Explores Sale, Hold...",2025-10-31,MT Newswires,4,3,3,https://finance.yahoo.com/news/market-chatter-...
9,Heard on the Street Recap: Market Streak,2025-11-01,The Wall Street Journal,5,5,0,https://finance.yahoo.com/m/c56c6928-c549-3d23...


## 11. Combined Analysis Function

In [67]:
def analyze_stock_news(ticker: str) -> List[Dict]:
    """
    Analyze stock news for a given ticker and return formatted results.
    
    Args:
        ticker: Stock ticker symbol
    
    Returns:
        List of dictionaries with news analysis
    """
    TOP_N_NEWS = 10
    MONTHS_BACK = 3
    
    # Fetch news
    news_items = get_stock_news(ticker, months_back=MONTHS_BACK)
    
    # Select interesting news
    interesting_news = select_interesting_news(news_items, ticker, top_n=TOP_N_NEWS)
    
    # Fetch full content
    news_with_content = fetch_news_content(ticker, interesting_news)
    
    # Score all news
    scored_news = score_all_news(news_with_content, ticker)
    
    # Format output
    result = []
    for item in scored_news:
        scores = item.get('scores', {})
        result.append({
            "nom": item['title'],
            "score": scores.get('sentiment_score', 5) / 2,
            "résumé": scores.get('explanation', ''),
            "lien": item['link'],
            "confiance": scores.get('likelihood_score', 5) / 2
        })
    
    return result

In [ ]:
# Test the combined function
# result = analyze_stock_news(STOCK_TICKER)
# print("Analysis complete! Here's the formatted output:")
# import json
# print(json.dumps(result, indent=2, ensure_ascii=False))

Fetching news for AAPL...
Found 894 news items from the last 3 months

Using Amazon Bedrock AI to select top 10 most interesting news items...
Selected 10 news items

Fetching full content for selected news...
Fetching content 1/10...
Fetching content 2/10...
Fetching content 3/10...
Fetching content 4/10...
Fetching content 5/10...
Fetching content 6/10...
Fetching content 7/10...
Fetching content 8/10...
Fetching content 9/10...
Fetching content 10/10...
Content fetching complete!

Scoring news items with Amazon Bedrock AI...
Scoring news 1/10...
Scoring news 2/10...
Scoring news 3/10...
Scoring news 4/10...
Scoring news 5/10...
Scoring news 6/10...
Scoring news 7/10...
Scoring news 8/10...
Scoring news 9/10...
Scoring news 10/10...
Scoring complete!
Analysis complete! Here's the formatted output:
[
  {
    "nom": "Apple announces additional $100 billion in US investment, following Trump iPhone tariff threat",
    "score": 3.5,
    "résumé": "A $100B US investment commitment is signi